# Training CNNs on CIFAR10

![alt text](https://miro.medium.com/max/1224/0*EL6Fy9lJicWeNY2z)


### Q1: Load the CIFAR10 data and visualize the first batch
a) Try some data augmentation

In [0]:
import torch

import matplotlib.pyplot as plt

from torchvision import datasets, transforms
  

batch_size = 32


##################  Change these lines  ####################################
# TODO: Try some data augmentation

transform_train = transforms.Compose([
    transforms.ToTensor()
])

transform_val = transforms.Compose([
    transforms.ToTensor()
])

##################  Finish changing here  ##################################


# datasets (CIFAR10)
datasets = {'train': datasets.CIFAR10('/data', train=True, download=True, transform=transform_train),
            'val': datasets.CIFAR10('/data', train=False, download=True, transform=transform_val)}

# dataloaders
loaders = {}
for phase in ['train', 'val']:
    loaders[phase] = torch.utils.data.DataLoader(datasets[phase], 
                                                 batch_size=batch_size, 
                                                 shuffle=True, 
                                                 drop_last=True)


def visualize_batch(batch, labels, ncols=8):
    nrows = (batch.shape[0] + ncols - 1) // ncols
    plt.figure(figsize=(15, 2*nrows))
    for i in range(batch.shape[0]):
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(batch[i].permute(1, 2, 0).squeeze(), interpolation='bilinear')
        plt.title(labels[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()


label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


batch, labels = next(iter(loaders['train']))
visualize_batch(batch, [label_names[x] for x in labels])

### Q2: Create the network
a) Try with different models and make your own architectures \\
b) Set the hyperparameters, the loss function and the optimizer

In [0]:
import torch.nn as nn
from torchvision import models


##################  Change these lines  ####################################
# TODO: Define the network architecture


model = ...


##################  Finish changing here  ##################################


# learning rate
lr = ...

# number of epochs
n_epochs = ...

# loss function
lossf = ...

# try other optimizer??
optimizer = ...

# set device to use
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)

### Q3: Train the network

a) Complete the code to compute the ouput of the network, the loss, to do the backprop and to compute an optimization step

![alt text](https://4.bp.blogspot.com/-Cu1mJOh11AU/XAIcUyPK0WI/AAAAAAAANNA/BRlNj0Cbt6EJHNH25D4RhB0e6_sbL1Y8QCLcBGAs/s640/28056576_10213577221682063_7572084637958860851_n.jpg)

In [0]:
import IPython.display


loss_history = {'train': [], 'val': []}
acc_history = {'train': [], 'val': []}

batch_loss = []
batch_acc = []


def plot_results(epoch, batch, total_batches):
    IPython.display.clear_output(wait=True)
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.title('Batch loss (epoch: %d, batch: %d/%d)' % (epoch, batch, total_batches))
    plt.plot(batch_loss)
    plt.yscale('log')

    plt.subplot(1, 4, 2)
    plt.title('Batch accuracy')
    plt.plot(batch_acc)

    plt.subplot(1, 4, 3)
    plt.title('loss')
    for phase in ['train', 'val']:
        plt.plot(loss_history[phase], label=phase)
        plt.yscale('log')
    plt.legend()

    plt.subplot(1, 4, 4)
    if len(acc_history['train']) > 0:
        plt.title('accuracy ' + ','.join(['%s: %.3f' % (phase, acc_history[phase][-1]) for phase in ['train', 'val']]))
    else: 
        plt.title('accuracy')

    for phase in ['train', 'val']:
        plt.plot(acc_history[phase], label=phase)
    plt.legend()

    plt.tight_layout()
    plt.show(block=False)


# number of batches in for the train and validation phases
num_batches = {'train': len(loaders['train']),
               'val': len(loaders['val'])}

# iterate on epochs
for i in range(n_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()  

        running_loss = 0.0
        running_acc = 0.0
        
        model.train()

        # iterate on batches
        for j, (samples, targets) in enumerate(loaders[phase]):
            samples, targets = samples.to(device), targets.to(device)
            
            ##################  Change these lines  ####################################
            # TODO: compute the ouput of the network, the loss, do the backprop and an 
            # optimization step





            ##################  Finish changing here  ##################################

            acc = torch.sum(outputs.max(dim=1)[1] == targets).item() / batch_size
            running_loss += loss.item()
            running_acc += acc
            
            if j % 50 == 0 and phase == 'train':
                # track batch history
                batch_loss.append(loss.item())
                batch_acc.append(acc)
                plot_results(i, j, num_batches[phase])

        running_loss /= num_batches[phase]
        running_acc /= num_batches[phase]
        loss_history[phase].append(running_loss)
        acc_history[phase].append(running_acc)

### Prediction examples

In [0]:
model.cpu()
samples, labels = next(iter(loaders['val']))
outputs = model(samples)
pred = outputs.max(dim=1)[1]
visualize_batch(samples, ['%s/%s' % (label_names[labels[i]], label_names[pred[i]]) for i in range(len(labels))])

### Confussion matrix

In [0]:
import numpy as np


model.to(device)
model.eval()
matrix = np.zeros(shape=(10, 10))
# iterate on batches
for samples, targets in loaders['val']:
  samples, targets = samples.to(device), targets.to(device)
  outputs = model(samples)
  pred = outputs.max(dim=1)[1]
  

  for i in range(targets.shape[0]):
      matrix[targets[i]][pred[i]] += 1

plt.imshow(matrix)

plt.xticks(range(10), label_names, rotation='vertical');
plt.yticks(range(10), label_names);

### Probabilities of some samples

In [0]:
import torch.nn.functional as F


batch, labels = next(iter(loaders['val']))
model.cpu()
model.eval()

for i in range(10):
    plt.figure(figsize=(8,2))
    test_image = batch[i]
    
    plt.subplot(1,2,1)
    plt.imshow(test_image.permute(1, 2, 0).squeeze(), interpolation='bilinear')
    plt.axis('off')

    plt.subplot(1,2,2)
    
    # Softmax to make the probabilities to be between 0 and 1
    p = F.softmax(model(test_image[None]).detach().squeeze())
    
    plt.bar(range(len(p)), p)
    plt.xticks(range(len(p)), label_names, rotation='vertical')
    plt.show()